<a href="https://colab.research.google.com/github/kglobe/python_ml/blob/master/twse%E4%BA%94%E6%AA%94%E6%8E%83%E7%B4%AF%E7%A9%8D%E6%88%90%E7%B8%BE%E9%87%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, clear_output, HTML
from urllib.request import urlopen
import pandas as pd
import datetime
import requests
import sched
import time
import json
import pytz

s = sched.scheduler(time.time, time.sleep)

def tableColor(val):
    if val > 0:
        color = 'red'
    elif val < 0:
        color = 'green'
    else:
        color = 'white'
    return 'color: %s' % color

def stock_crawler(tse_targets,txpe_targets):
  clear_output(wait=True)
      
  # 組成stock_list
  tse_stock_list = '|'.join('tse_{}.tw'.format(target) for target in tse_targets)
  txpe_stock_list = '|'.join('otc_{}.tw'.format(target) for target in txpe_targets)

  #　query data
  query_url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch="+ tse_stock_list+'|'+txpe_stock_list
  print('query_url:',query_url)
  # data = json.loads(urlopen(query_url).read())
  resp = requests.get(query_url)
  data = resp.json()

  # 過濾出有用到的欄位
  columns = ['c','n','z','tv','v','o','h','l','y']
  df = pd.DataFrame(data['msgArray'], columns=columns)
  df.columns = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
  df.insert(9, "漲跌百分比", 0.0) 
  df = df.replace('-',0)
  # 新增漲跌百分比
  for x in range(len(df.index)):
    if df['當盤成交價'].iloc[x] != '-':
        df.iloc[x, [2,3,4,5,6,7,8]] = df.iloc[x, [2,3,4,5,6,7,8]].astype(float)
        df['漲跌百分比'].iloc[x] = (df['當盤成交價'].iloc[x] - df['昨收價'].iloc[x])/df['昨收價'].iloc[x] * 100

  # 紀錄更新時間
  now = datetime.datetime.now(pytz.timezone('Asia/Taipei'))
  print("更新時間:" + now.strftime('%Y-%m-%d %H:%M:%S'))

  # show table
  df = df.style.applymap(tableColor, subset=['漲跌百分比'])
  display(df)

  # start_time = datetime.datetime.strptime(str(time.date())+'9:30', '%Y-%m-%d%H:%M')
  # end_time =  datetime.datetime.strptime(str(time.date())+'13:30', '%Y-%m-%d%H:%M')
  start_time = datetime.datetime.strptime(str(now.date())+'00:30+0800', '%Y-%m-%d%H:%M%z')
  end_time =  datetime.datetime.strptime(str(now.date())+'13:30+0800', '%Y-%m-%d%H:%M%z')
  # 判斷爬蟲終止條件
  if now >= start_time and now <= end_time:
      s.enter(5, 0, stock_crawler, argument=(tse_targets,txpe_targets,))

# 欲爬取的股票代碼
twse_stock_list = ['1101','1102','1103','2002']
txpe_stock_list = ['1586','1788','3064']

# 每秒定時器
s.enter(5, 0, stock_crawler, argument=(twse_stock_list,txpe_stock_list,))
s.run()